In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
from scipy.spatial.distance import pdist, squareform

In [2]:
genres_recommender = pd.read_pickle("Data_Hulu_Disney/genres_recommender_v2.pkl")

survey_shows = pd.read_pickle("Data_Hulu_Disney/survey_shows.pkl")

In [3]:
test = pd.read_csv("C:/Users/Admin/Downloads/Untitled spreadsheet - Sheet2.csv")

In [4]:
test

,Timestamp,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,...,Question 22,Question 23,Question 24,Question 25,Question 26,Question 27,Question 28,Question 29,Question 30,Name
0,5/20/2020 8:37:29,Revisions (2019) - When Shibuya time-warps to ...,None,One Strange Rock (2018) - The extraordinary st...,Zoey's Extraordinary Playlist (2020) - A young...,The Joy of Painting (1983) - In this half-hour...,Black Mirror (2011) - An anthology series expl...,Stranger Things (2016) - When a young boy disa...,Crazy Ex-Girlfriend (2015) - A young woman aba...,How I Met Your Mother (2005) - A father recoun...,...,America's Funniest Home Videos (1989) - Viewer...,Just Add Magic (2015) - Three friends cook up ...,Happy Endings (2011) - This Chicago-set sitcom...,The Imagineering Story (2019) - The inside loo...,K.C. Undercover (2015) - An outspoken and conf...,Encore! (2019) - Executive producer Kristen Be...,Good Omens (2019) - A tale of the bungling of ...,The Simpsons (1989) - The satiric adventures o...,Top Chef (2006) - A reality competition show i...,Tian


In [5]:
def get_recommender_df(survey_df, df_all_shows):
    
    while True:
        
        name = input("Please enter your name:")
        
        try:

            if name in survey_df["Name"].values:
                break;
            
            else:
                print("That user name does not match. Please try again.")
                
        except ValueError:
            print("That user name does not match. Please try again.")

    user_df = survey_df.loc[np.where(survey_df["Name"] == name)]
    
    user_df["Timestamp"] = pd.to_datetime(user_df["Timestamp"])
    
    user_df = user_df.iloc[np.where(user_df["Timestamp"] == user_df["Timestamp"].max())]
    
    user_df = user_df.drop(columns=["Timestamp", "Name"])
   
    genre_columns = ['Crime', 'Drama', 'Thriller', 'Fantasy', 'Horror', 'Mystery', 'Comedy', 'Sci-Fi', 'Biography',
                 'Action', 'Adventure', 'Romance', 'History', 'Documentary', 'Animation', 'War', 'Sport',
                 'Family', 'Western', 'Short', 'Reality-TV', 'Musical', 'Music', 'Game-Show', 'Talk-Show', 'News']
        
    all_shows_lst = [show for show in df_all_shows["show"]]
    
    titles = []
    
    # we will create a list for the TV shows selected, excluding 'None' answers, and  we will iterate through this list
    # to get the titles
    user_shows = [show for show in user_df.values[0] if show != "None"]
        
    # we will select just the last element using the tail pandas method
    for show in user_shows:
        # remove the year and description
        title_pattern = r"(.*)\s\(\d{4}\).*$"
        title = re.findall(title_pattern, show)
        titles.append(title[0])

    matches = []

    for title in titles:
        for show in all_shows_lst:
            ratio = fuzz.ratio(title.lower(), show.lower())
            if ratio == 100:
                matches.append(show)
                matches = list(set(matches))
                              
    user_df = df_all_shows[df_all_shows["show"].isin(matches)].reset_index(drop=True)
    
    user_genre_ratio = (pd.DataFrame(user_df[genre_columns].sum()
                                    .sort_values(ascending=False))
                                   .reset_index()
                                   .rename(columns={"index":"genre", 0:"frequency"}))
    
    # since the shows selected may vary, we will divide it by the length of the user_shows list to get the ratio
    user_genre_ratio["ratio"] = round((user_genre_ratio["frequency"] / len(user_shows)), 4)

    user_genre_ratio["user"] = "User"

    user_recommender = user_genre_ratio.pivot_table(index="user",
                                                  columns="genre",
                                                  values="ratio")
    
    return user_recommender, name

In [ ]:
df, name = get_recommender_df(test, survey_shows)

Please enter your name:Ilse
That user name does not match. Please try again.
Please enter your name:ilse
That user name does not match. Please try again.


In [12]:
df

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,
User,0.1667,0.0667,0.1333,0.0,0.6,0.1,0.2,0.1667,0.3667,0.1,...,0.0,0.2667,0.1,0.1,0.0,0.0,0.0,0.0667,0.0,0.0
